# **Lab 10 - Explainable and Trustworthy AI**


---



**Teacher**: Eliana Pastor (eliana.pastor@polito.it)


---

## **Counterfactual explanatios**

We generate counterfactual of individual predictions provided.
We consider the [DICE](https://dl.acm.org/doi/10.1145/3351095.3372850) algorithm and the [dice-ml](https://interpret.ml/DiCE/) library.
Part of this laboratory is adapted from the official examples of the library. You can refer to the original notebook examples from full details on the library and its usage.


We focus on the [Adult dataset](https://archive.ics.uci.edu/dataset/2/adult) and a Random Forest classifier, as done for the previous labs (e.g., 3b and 7a).

> **Dataset.** The Adult dataset, also known as the "Census Income" dataset, contains demographic information about people, such as age, education, occupation, marital status and more, extracted from the 1994 U.S. Census Bureau database. **Each entry** in the dataset represents a **person**, and the associated **task** is to **predict whether an individual earns more than $50,000 per year** or less.

> **Model**

* We first load the Adult dataset.
  * We can directly [load the dataset from UCI](https://archive.ics.uci.edu/dataset/2/adult) and process it. The dataset is available from the UCI repository. Using the library ucimlrepo we can easily fetch the dataset. Alternatively, we can use the dataset available in the SHAP [library](https://shap.readthedocs.io/en/latest/generated/shap.datasets.adult.html) as done for the previous labs.
  Or w
*  We split the Adult dataset. 80/20 train-test ratio.
*  We then rain a RandomForestClassifier and fit it over the training dataset. Evaluate the model.


> **Counterfactual Explanations**

Use the algorithm DICE to generate counterfactual.

1. Generate counterfarfactuals for the instance `id=0` of the test set using defaul parameters.
2. Generate counterfarfactuals imposing a actionability constraint. Specify the set of attributes to be modified to generate the counterfactuals.
3. Generate counterfarfactuals imposing a feasibility constraints. Specify the range of values of the features.
4. DICE computes a feature importance considering the number of times a feature is changed to generate a counterfactual.
The more ofter a feature is changed, the more it is important to generate the counterfactuals.

In [ ]:
! pip install ucimlrepo

In [ ]:
!pip install dice-ml

In [ ]:
%load_ext autoreload
%autoreload 2

# Data import, processing and training

In the following, first load the data, preprocess it and train a Random Forest classifier

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets.copy() 
y.replace({'<=50K.': '<=50K', '>50K.': '>50K'}, inplace=True)

In [ ]:
attributes = list(X.columns)
target_class = list(y.columns)[0]

Split train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

Preprocessing

In [ ]:
X.describe()

In [ ]:
numerical = ['age', 'fnlwgt', 'hours-per-week', 'education-num', 'capital-gain', 'capital-loss']
categorical = X_train.columns.difference(numerical)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

In [ ]:
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(X_train, y_train)

# Counterfactual explanantions using DICE

Import DICE package

In [ ]:
import dice_ml

In [ ]:
# Provide the trained model to DiCE's model object
# Note that we specify tha the model is a sklearn model
backend = 'sklearn'
m = dice_ml.Model(model=model, backend=backend)

In [ ]:
df_data = X_train.copy()
df_data[target_class] = y_train.values

# Define the data to generate counterfactuals
# We specify the data (target class included), the continuous features and the target class

d = dice_ml.Data(dataframe=df_data, \
                 continuous_features=numerical, 
                 outcome_name=target_class)


DICE supports multiple algorithms to generate counterfactuals. We use the 'random' method based on random sampling of features

In [ ]:
dice_alg = dice_ml.Dice(d, m, method="random")

## 1. Generate counterfactuals for a single instance

Select the instance for which we want to generate a counterfactual. 

We select the first instance of the test set

In [ ]:
# Specify the query instance for which we want to generate the counterfactual
# Use the first instance of the test set
# We suggest to specify it as a pandas dataframe
query_instance =  # Add the query instance here
query_instance

What is the original prediction for the query instance?

In [ ]:
# Predict the class for the query instance
# 

Generate 3 counterfactuals for the query instance

In [ ]:
# Generate the counterfactuals
dice_exp = dice_alg.generate_counterfactuals(query_instance, 
                                             total_CFs= ###, #Specify the number of counterfactuals to generate
                                             desired_class="opposite", # We want to generate counterfactuals that will have as predicted class the opposite of the original instance
                                             verbose=False, 
                                             random_seed=7)

You can visualize the counterfactuals using the visualize_as_dataframe method. This method will return a pandas dataframe with the counterfactuals generated.

By setting show_only_changes=True, it will generate a sparse visualization of the counterfactuals, were only the changed features and values are reported

In [ ]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Get the counterfactuals as a dataframe

In [ ]:
counterfactuals_df = dice_exp.cf_examples_list[0].final_cfs_df
counterfactuals_df

Check that indeed the model predicts the opposite class for the counterfactuals

In [ ]:
# Predict the class for the counterfactuals

## 2. Actionable Counterfactuals

Goal: Generate counterfarfactuals imposing a actionability constraint.
We specify the set of attributes to be modified to generate the counterfactuals.

Depending on the data and task, it is not always possible to modify some attributes.append
For example, if we want to generate a counterfactual for a person with a certain age, we cannot change the age of the person. 
In this case, we can specify the attributes that we want to keep fixed using the features_to_vary parameter.

We can use the parameter `features_to_vary` to specify the features that we want to vary

In [ ]:
# generate counterfactuals
dice_exp = dice_alg.generate_counterfactuals(
        query_instance, 
        total_CFs= ##, #Specify the number of counterfactuals to generate
        desired_class="opposite",
        features_to_vary= ###, #Specify the features to vary
        random_seed=7)

In [ ]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

## 3. Feasible Counterfactuals

Generate counterfarfactuals imposing a feasibility constraints. 
In generate counterfactuals, we should also consider the range the feature values can assume.
For example, we cannot increase indiscriminately the number of hours per week or the age range. 

In DICE, we specify the admitted range of values of the features.

We can use the parameter `permitted_range` to specify the range admitted for set of features. We specify the permitted range as a dictionary: {'attribute_name' :  [min_value, max_value], ..}

In [ ]:
# generate counterfactuals
dice_exp = dice_alg.generate_counterfactuals(
    query_instance, 
    total_CFs= ##3, # Specify the number of counterfactuals to generate
    desired_class="opposite", 
    permitted_range= ###) # Specify the permitted range for the features as a dictionary : {feature_name: [min_value, max_value], ..}

In [ ]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

## 4. Feature importance for the counterfactuals

DICE computes a feature importance considering the number of times a feature is changed to generate a counterfactual.
The more ofter a feature is changed, the more it is important to generate the counterfactuals.

For the computation of feature importance we use the method `local_feature_importance`.
The method needs a set of counterfactuals (minimum 10) generated for the same query instance and computes the feature importance.

In [ ]:
# Generate the counterfactuals
dice_exp = dice_alg.generate_counterfactuals(query_instance, 
                                             total_CFs= ##, #Specify the number of counterfactuals to generate (>10)
                                             desired_class="opposite", # We want to generate counterfactuals that will have as predicted class the opposite of the original instance
                                             verbose=False, 
                                             random_seed=7)

In [ ]:
# Generate the feature importance
 
dice_exp_loc_imp = dice_alg.local_feature_importance(query_instance, 
                                             cf_examples_list=dice_exp.cf_examples_list)

In [ ]:
# Print the local feature impornance
 
## 

Plot the local feature importance as a bar chart

In [ ]:
# Plot the feature importance